In [1]:
from Crawler import Crawler

# tool
import pandas as pd
import numpy as np
import tqdm
import pickle
import math
import time
import random
import re
import matplotlib.pylab as plt

## sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import pairwise

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Bert
from transformers import *

from rouge_score import rouge_scorer
from rouge import Rouge 

In [2]:
crawler = Crawler()

In [3]:
train_csv = pd.read_csv('train.csv')

In [4]:
train_csv.groupby('domain').count()

,news_ID,hyperlink,content,name,article
domain,,,,,
ccc.technews.tw,10,10,10,10,10
domestic.judicial.gov.tw,30,30,30,30,30
ec.ltn.com.tw,6,6,6,6,6
ent.ltn.com.tw,2,2,2,2,2
estate.ltn.com.tw,1,1,1,1,1
finance.technews.tw,56,56,56,56,56
hk.on.cc,57,57,57,57,57
house.ettoday.net,1,1,1,1,1
m.ctee.com.tw,530,530,530,530,530


In [9]:
train_csv['start'] = train_csv.content.apply(lambda x: x.split('###')[0][:-1])
train_csv['end']   = train_csv.content.apply(lambda x: x.split('###')[2][1:])

## news.cnyes.com

In [10]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
rouge  = Rouge()

In [31]:
def detail(df, domain):
    df_c = crawler.get_domain_dataframe(df, domain)

    for index in df_c.index[:100]:
        
        content = df.loc[index].content
        start   = df.loc[index].start
        end     = df.loc[index].end
        article = df.loc[index].article
        
        score = scorer.score(start+end, article)['rouge1'].recall
        
        if score > 0.1:
            continue
            
        print('='*5, index, '='*5)
        print(score, '\n')
        print(df.loc[index].hyperlink, '\n')
        print(start, '\n')
        print(end, '\n')
        print(article, '\n')

In [30]:
detail(train_csv, 'news.cnyes.com')

===== 0 =====
0.5714285714285714 

https://news.cnyes.com/news/id/4352432 

0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35facebook commentFONT SIZEICON PRINT78981DD6-B6FA-435C-B969-0D6ABD01805C0「時間序列與量化交易研討會」邀集專家學者進行量化交易、智能投資等相關議題研討。(圖:鄭心芸攝)相關個股元大期6023近年來投資市場波動越來越明顯 

都有非常巨大的助益。 

理財基金量化交易追求絕對報酬 有效對抗牛熊市鉅亨網記者 鄭心芸2019/07/05 22:35facebook commentFONT SIZEICON PRINT近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易備受注目。專家表示，採用量化交易策略投資台股，不管是處於多頭或是空頭市場，績效及波動度均可領跑大盤，甚至比國內投資台股的股票型基金及 ETF 的波動率還低，表現也更為穩定。大數據時代來臨，風行歐美 50 年的量化交易儼然成為顯學，台灣亦開始重視此一趨勢發展，也因此，中華機率統計學會及台北科技大學管理學院攜手主辦，並由元大期貨、摩根亞太量化交易等公司擔任協辦單位，今 (7/5) 日舉辦「時間序列與量化交易研討會」，就目前熱門的量化交易、智能投資等相關議題進行研討。越來越多的基金公司重視量化交易，全球規模較大的避險基金多採行量化交易，包括橋水基金 (Bridgewater Associates)、AQR 資產管理公司、曼氏集團 (Man Group)、文藝復興科技 (Renaissance Technologies) 等全球知名避險基金。摩根亞太集團董事長張堯勇指出，避險基金規模約為 5 兆美元，採取量化交易的基金規模約 1 兆美元，比重佔了 20%，代表量化交易的操作績效好，才會有那麼高的比重。量化交易的操作績效不亞於價值投資及技術投資，被譽為數學天才、最賺錢的基金經理人，文藝復興對沖基金創始人詹姆斯 · 西蒙斯 (James Simons) 所管理的大獎章 (Medallion) 基金，便是典型的量化交易，績效表現優異，不僅勝過索羅斯的量子基金，也打敗了股神巴菲特的價值投資。近年來台灣也逐漸重

In [18]:
train_csv_no_404 = train_csv[train_csv.article != '文章已被刪除 404 or 例外']\
.drop(['hyperlink', 'domain'], axis=1)

In [87]:
train_csv_no_404.head(5)

,news_ID,content,name,article
0,1,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...,[],理財基金量化交易追求絕對報酬 有效對抗牛熊市鉅亨網記者 鄭心芸2019/07/05 22:3...
1,2,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ### 省略內文 ### 北...,[],10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件，北市府勞動局認定業者未依職業...
2,3,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文...,[],社會2019.10.08 09:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從...
4,5,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...,[],例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...
5,6,內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區 ### 省略內文 ### 市區拒...,[],內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區，將於8月30日到高雄大社現勘（註...


In [159]:
names_list = list(train_csv_no_404.name.values)
documents  = list(train_csv_no_404.article.values)
contents   = list(train_csv_no_404.content.values)

In [160]:
for names, document in zip(names_list, documents):
    names_chosen = ''
    
    if names != '[]':
        names = names.replace('\'', '').replace(']', '').replace('[', '').replace(' ', '').split(',')
        new_document = document[:]
        
        for name in names:
            if name == 'AlanLim':
                name = 'Alan Lim'
            if name == 'MynusLee':
                name = 'Mynus Lee'
            try:
                b_index = int(new_document.index(name))
                e_index = b_index+len(name)
                
                new_document = new_document[:b_index]+'[SEP]'+new_document[b_index: e_index]+\
                '[SEP]'+new_document[e_index:]
                
            except Exception as e:
                print(e)
                pass

#         print(new_document, '\n')
    else:
        continue

substring not found
substring not found
substring not found
substring not found
substring not found
